In [1]:
import numpy as np
import pandas as pd
import os, sys

sys.path.append("..")
print(sys.path)
from settings import  img_cache_path, DATA_DIR
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader
import torch
import time

['/home/kailu/deepfake', '/home/kailu/.conda/envs/wukailu/lib/python37.zip', '/home/kailu/.conda/envs/wukailu/lib/python3.7', '/home/kailu/.conda/envs/wukailu/lib/python3.7/lib-dynload', '', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages/googledrivedownloader-0.4-py3.7.egg', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages/rdflib-4.2.2-py3.7.egg', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages/plyfile-0.7.1-py3.7.egg', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages/isodate-0.6.0-py3.7.egg', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages/torch_geometric-1.3.2-py3.7.egg', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages/dlib-19.19.99-py3.7-linux-x86_64.egg', '/home/kailu/.conda/envs/wukailu/lib/python3.7/site-packages/IPython/extensions', '/home/kailu/.ipython', '..']


In [2]:
train_sample_metadata = pd.read_json(os.path.join(DATA_DIR,'metadata_all.json') ).T
indexes = train_sample_metadata[train_sample_metadata['label']=='REAL'].index

In [5]:
import pathlib
from utils.engine import VideoDataset, FullVideoReader, MTCNN_extractor, show
import pdb

def extract_imgs(index, sample_rate, exist_ok = False):
    print("index from:" , index[0], "to:", index[-1])
    ds = VideoDataset([os.path.join(DATA_DIR, fn) for fn in index], sample_rate = sample_rate, video_reader=FullVideoReader())
    loader = DataLoader(ds, batch_size=1, num_workers=20, drop_last=False, shuffle=False)
    
    iter_index = iter(index)
    for batch in tqdm(loader):
        batch = batch.data.numpy().astype(np.uint8)
        for frames in batch:
            fn = next(iter_index)
            if len(frames) == 0:
                print("an empty frame")
                continue
            cache_path = pathlib.Path(img_cache_path) / fn.split('.')[0]
            cache_path.mkdir(exist_ok=True)

            iter_frame = np.linspace(0, 300 - 1, sample_rate).round().astype(int)
            for j in range(len(iter_frame)):
                fid = iter_frame[j]
                target_path = cache_path / (str(fid) + '.png')
                if target_path.exists() and exist_ok:
                    continue
                img = Image.fromarray(frames[j])
                img = img.resize(np.array(img.size)//2)
                img.save(target_path, compress_level=1)

In [22]:
def skip_existed(indexes):
    ret = []
    for fn in indexes:
        cache_path = pathlib.Path(img_cache_path) / fn.split('.')[0]
        checks = np.linspace(0, 300 - 1, 13).round().astype(int)
        for ck in checks:
            target_path = cache_path / (str(ck) + '.png')
            if not target_path.exists():
                ret.append(fn)
                break
    print(len(indexes), len(ret))
    return ret

In [23]:
index = skip_existed(train_sample_metadata.index)

119154 9


In [21]:
extract_imgs(index, sample_rate=13)


  0%|          | 0/4645 [00:00<?, ?it/s]

index from: alnkzqihau.mp4 to: zubaiztkag.mp4



  3%|▎         | 149/4645 [04:16<1:40:59,  1.35s/it]


  6%|▋         | 295/4645 [08:20<2:05:14,  1.73s/it]


  9%|▉         | 441/4645 [12:15<55:30,  1.26it/s]


...

 89%|████████▉ | 4150/4645 [1:35:33<12:05,  1.47s/it]


 92%|█████████▏| 4295/4645 [1:38:42<05:03,  1.15it/s]


 97%|█████████▋| 4484/4645 [1:41:53<04:34,  1.71s/it]


100%|█████████▉| 4627/4645 [1:44:59<00:10,  1.78it/s]


100%|██████████| 4645/4645 [1:45:09<00:00,  2.16it/s]


In [27]:
list(set([0, 25, 50, 75, 125, 150]))

[0, 75, 50, 150, 25, 125]